#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

# 導入必要的函數

In [1]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

Using TensorFlow backend.


# 資料準備

In [2]:
#取得Keras Dataset
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [3]:
#確認 CIFAR10 Dataset 資料維度
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) 

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
#資料正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

In [5]:
#針對Label 做 ONE HOT ENCODE
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape

(10000, 10)

# 建立模型

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [7]:
model = Sequential()

In [8]:
#卷積層1

In [9]:
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))

In [10]:
model.add(Dropout(rate=0.25))

In [11]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [12]:
#卷積層2與池化層2

In [13]:
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))

In [14]:
model.add(Dropout(0.25))

In [15]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [16]:
#建立神經網路(平坦層、隱藏層、輸出層)

In [17]:
model.add(Flatten())
model.add(Dropout(rate=0.25))

In [18]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.25))

In [19]:
model.add(Dense(10, activation='sigmoid'))

In [20]:
#檢查model 的STACK
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)             

# 載入之前訓練的模型

In [21]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

載入模型失敗!開始訓練一個新模型


# 訓練模型

In [22]:
# final layer use softmax
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['categorical_accuracy'])
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)    
'''
作業:
請分別選用 "MSE", "binary_crossentropy"
查看Train/test accurancy and loss rate
'''

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 58s 2ms/step - loss: 2.1653 - categorical_accuracy: 0.2014 - val_loss: 2.0548 - val_categorical_accuracy: 0.2970
Epoch 2/12
37500/37500 [==============================] - 63s 2ms/step - loss: 1.9511 - categorical_accuracy: 0.2992 - val_loss: 1.9526 - val_categorical_accuracy: 0.3377
Epoch 3/12
37500/37500 [==============================] - 66s 2ms/step - loss: 1.8251 - categorical_accuracy: 0.3503 - val_loss: 1.8232 - val_categorical_accuracy: 0.3955
Epoch 4/12
37500/37500 [==============================] - 73s 2ms/step - loss: 1.7228 - categorical_accuracy: 0.3863 - val_loss: 1.7425 - val_categorical_accuracy: 0.4192
Epoch 5/12
37500/37500 [==============================] - 82s 2ms/step - loss: 1.6391 - categorical_accuracy: 0.4164 - val_loss: 1.6564 - val_categorical_accuracy: 0.4530
Epoch 6/12
37500/37500 [==============================] - 86s 2ms/step - loss: 1.5693 - categor

'\n作業:\n請分別選用 "MSE", "binary_crossentropy"\n查看Train/test accurancy and loss rate\n'

In [23]:
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print("categorical_crossentropy")
print('accuracy=',scores[1])
model.save("SaveModel/categorical_crossentropy.h5")

10000/10000 [==============================] - 6s 554us/step
categorical_crossentropy
accuracy= 0.5468999743461609


In [24]:
print(model.predict(x_img_test_normalize[0:2]))
print(y_label_test_OneHot[0:2])


[[0.01348561 0.00598503 0.09741496 0.40280977 0.04185493 0.27953488
  0.11722513 0.01215899 0.02324933 0.00628138]
 [0.12977049 0.3912384  0.00618166 0.00184511 0.00280236 0.00042548
  0.0020965  0.00087987 0.35142323 0.11333689]]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [21]:
# final layer use sigmoid
model.compile(loss= 'mean_squared_error', optimizer='sgd', metrics=['categorical_accuracy'])
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 95s 3ms/step - loss: 0.1073 - categorical_accuracy: 0.1025 - val_loss: 0.0986 - val_categorical_accuracy: 0.1048
Epoch 2/12
37500/37500 [==============================] - 92s 2ms/step - loss: 0.0927 - categorical_accuracy: 0.1067 - val_loss: 0.0979 - val_categorical_accuracy: 0.1187
Epoch 3/12
37500/37500 [==============================] - 86s 2ms/step - loss: 0.0924 - categorical_accuracy: 0.1134 - val_loss: 0.0978 - val_categorical_accuracy: 0.1440
Epoch 4/12
37500/37500 [==============================] - 86s 2ms/step - loss: 0.0922 - categorical_accuracy: 0.1170 - val_loss: 0.0980 - val_categorical_accuracy: 0.1522
Epoch 5/12
37500/37500 [==============================] - 87s 2ms/step - loss: 0.0920 - categorical_accuracy: 0.1220 - val_loss: 0.0979 - val_categorical_accuracy: 0.1564
Epoch 6/12
37500/37500 [==============================] - 89s 2ms/step - loss: 0.0918 - categor

In [22]:
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print("mean_squared_error")
print('accuracy=',scores[1])
model.save("SaveModel/mean_squared_error.h5")

10000/10000 [==============================] - 5s 517us/step
mean_squared_error
accuracy= 0.18520000576972961


In [23]:
# final layer use sigmoid
model.compile(loss= 'binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 95s 3ms/step - loss: 0.2543 - categorical_accuracy: 0.3999 - val_loss: 0.2439 - val_categorical_accuracy: 0.5303
Epoch 2/12
37500/37500 [==============================] - 94s 3ms/step - loss: 0.2003 - categorical_accuracy: 0.5552 - val_loss: 0.2015 - val_categorical_accuracy: 0.5954
Epoch 3/12
37500/37500 [==============================] - 94s 3ms/step - loss: 0.1786 - categorical_accuracy: 0.6125 - val_loss: 0.1838 - val_categorical_accuracy: 0.6442
Epoch 4/12
37500/37500 [==============================] - 96s 3ms/step - loss: 0.1630 - categorical_accuracy: 0.6535 - val_loss: 0.1722 - val_categorical_accuracy: 0.6722
Epoch 5/12
37500/37500 [==============================] - 95s 3ms/step - loss: 0.1513 - categorical_accuracy: 0.6826 - val_loss: 0.1639 - val_categorical_accuracy: 0.6893
Epoch 6/12
37500/37500 [==============================] - 96s 3ms/step - loss: 0.1420 - categor

In [24]:
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print("binary_crossentropy")
print('accuracy=',scores[1])
model.save("SaveModel/binary_crossentropy.h5")

10000/10000 [==============================] - 6s 567us/step
binary_crossentropy
accuracy= 0.7452999949455261


In [25]:
prediction = model.predict(x_img_test_normalize)
tp = 0
for i, pre in enumerate(prediction):
    if np.argmax(y_label_test_OneHot[i]) == np.argmax(pre):
        tp+=1
print(tp/len(prediction))


0.7453


- binary_crossentropy要搭配sigmoid
- categorical_crossentropy要搭配softmax
- 多分類情況下，要採用metrics=['categorical_accuracy']